In [7]:
import numpy as np
import pandas as pd

# Load data from CSV
data = pd.read_csv('data_export1.csv')
# Extract relevant columns
sim_time = data['sim_time'].values
clean_position = data['clean_position'].values
clean_velocity = data['clean_velocity'].values
dirty_position = data['dirty_position'].values
dirty_velocity = data['dirty_velocity'].values

# State transition matrix
F = np.array([[1, 0.032], [0, 1]])

# Measurement matrix
H = np.array([[1, 0], [0, 1]])

# Process noise covariance
Q = np.array([[0.1, 0], [0, 0.1]])

# Measurement noise covariance
R = np.array([[0.1, 0], [0, 0.1]])

# Initialize Kalman filter state and covariance
x_hat = np.array([clean_position[0], clean_velocity[0]])
P = np.eye(2)

# Kalman filter
x_est = []
for i in range(len(sim_time)):
    # Prediction step
    x_pred = F @ x_hat
    P_pred = F @ P @ F.T + Q

    # Measurement update step
    y = np.array([dirty_position[i], dirty_velocity[i]])
    S = H @ P_pred @ H.T + R
    K = P_pred @ H.T @ np.linalg.inv(S)
    x_hat = x_pred + K @ (y - H @ x_pred)
    P = (np.eye(2) - K @ H) @ P_pred

    x_est.append(x_hat)

# Convert x_est to numpy array
x_est = np.array(x_est)

# Extract the estimated position and velocity
estimated_position = x_est[:, 0]
estimated_velocity = x_est[:, 1]

# Calculate the covariance matrix
cov_matrix = np.cov(estimated_position, estimated_velocity)

print("Covariance Matrix:")
print(cov_matrix)

# Calculate mean error and mean squared error
mean_error_clean_position = np.mean(clean_position - estimated_position)
mean_error_clean_velocity = np.mean(clean_velocity - estimated_velocity)
mean_error_dirty_position = np.mean(dirty_position - estimated_position)
mean_error_dirty_velocity = np.mean(dirty_velocity - estimated_velocity)

mse_clean_position = np.mean((clean_position - estimated_position)**2)
mse_clean_velocity = np.mean((clean_velocity - estimated_velocity)**2)
mse_dirty_position = np.mean((dirty_position - estimated_position)**2)
mse_dirty_velocity = np.mean((dirty_velocity - estimated_velocity)**2)

print("\nMean Error:")
print(f"Clean Position: {mean_error_clean_position:.4f}")
print(f"Clean Velocity: {mean_error_clean_velocity:.4f}")
print(f"Dirty Position: {mean_error_dirty_position:.4f}")
print(f"Dirty Velocity: {mean_error_dirty_velocity:.4f}")

print("\nMean Squared Error:")
print(f"Clean Position: {mse_clean_position:.4f}")
print(f"Clean Velocity: {mse_clean_velocity:.4f}")
print(f"Dirty Position: {mse_dirty_position:.4f}")
print(f"Dirty Velocity: {mse_dirty_velocity:.4f}")

Covariance Matrix:
[[ 0.03400683 -0.5224407 ]
 [-0.5224407  12.99305129]]

Mean Error:
Clean Position: -0.0403
Clean Velocity: -0.0030
Dirty Position: -0.0231
Dirty Velocity: -0.0023

Mean Squared Error:
Clean Position: 0.0138
Clean Velocity: 4.0223
Dirty Position: 0.0096
Dirty Velocity: 4.0154
